In [5]:
import torch
import torch.nn as nn
import numpy as np
import math
import tqdm

In [6]:
with open("data/sonar.all-data") as all_data_file:
    lines = all_data_file.readlines()
    all_data = []
    labels = []
    for line in lines:
        line = line.strip().split(',')
        label = line.pop()
        #line = np.asarray(line, dtype=float)
        if label == "R":
            labels.append(0)
            all_data.append(line)
        elif label == "M":
            labels.append(1)
            all_data.append(line)
        else:
            pass
    all_data = np.asarray(all_data, dtype=float)
    labels = np.asarray(labels, dtype=float)


In [7]:
def normalise_2darray(d2array):
    output_array = []
    for array in d2array:
        x = (array - np.mean(array)) / np.std(array)
        x[x<0] *= -1
        x = (x-np.min(x))/(np.max(x) - np.min(x))
        output_array.append(x)
    return np.asarray(output_array)

In [8]:
def train_test_split(input_data, input_labels):
    indices = np.random.permutation(input_data.shape[0])
    split_idx = math.floor(input_data.shape[0] * 0.7)
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    train_data, test_data = input_data[train_idx,:], input_data[test_idx,:]
    train_labels, test_labels = input_labels[train_idx], input_labels[test_idx]
    return train_data, test_data, train_labels, test_labels

In [9]:
x_norm = normalise_2darray(all_data)

corr_arr = []
for idx, row in enumerate(x_norm):
    arr = np.asarray(list(row) + [labels[idx]])
    corr_arr.append(arr)
corr_arr = np.asarray(corr_arr)
corr_map = np.corrcoef(corr_arr, rowvar=False).round(2)
corr_map = corr_map[:, 60]  # keep only final column of the heatmap | correlation to target class
corr_map = corr_map.reshape((61, 1))

to_drop = []
for idx, value in enumerate(corr_map):
    if value > -0.1 and value < 0.1:
        to_drop.append(idx)

x_norm = np.delete(x_norm, to_drop, axis=1)
x_norm.shape


(208, 35)

In [10]:
train_data, test_data, train_labels, test_labels = train_test_split(x_norm, labels)
print(train_data.shape, test_data.shape, train_labels.shape, test_data.shape)


(145, 35) (63, 35) (145,) (63, 35)


In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [12]:
train_data = torch.tensor(train_data, dtype=torch.float32).to(device)
test_data = torch.tensor(test_data, dtype=torch.float32).to(device)
train_labels = torch.tensor(train_labels, dtype=torch.float32).reshape(-1,1).to(device)
test_labels = torch.tensor(test_labels, dtype=torch.float32).reshape(-1,1).to(device)

print(train_data.shape, test_data.shape, train_labels.shape, test_labels.shape)

torch.Size([145, 35]) torch.Size([63, 35]) torch.Size([145, 1]) torch.Size([63, 1])


In [30]:
class Network(nn.Module):
    def __init__(self, input_size, output_size) -> None:
        super(Network, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.network = self.__setup_network()

    def __setup_network(self):
        net = nn.Sequential(
            nn.Linear(self.input_size, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Sigmoid(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, self.output_size),
            nn.Sigmoid(),
        )
        return net
    def forward(self, input):
        return self.network(input)
    
    def fit(self, X, y, n_epochs, loss_fn, optimizer, batch_size=30):
        batches = torch.arange(0, len(X), batch_size)
        for epoch in range(n_epochs):
            best_acc = 0
            best_loss = 0
            for batch_start in batches:
                X_batch = X[batch_start:(batch_start+batch_size)]
                y_batch = y[batch_start:(batch_start+batch_size)]

                prediction = self.forward(X_batch)
                loss = loss_fn(prediction, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                acc = (prediction.round() == y_batch).float().mean()

                best_acc = acc if acc > best_acc else best_acc
                best_loss = loss if loss < best_loss else best_loss

            print(f"Epoch {epoch}:\t loss(best per batches): {loss} accuracy(best per batch): {acc}")

            
    


In [34]:
model = Network(input_size=35, output_size=1).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.RMSprop(model.network.parameters(), lr=0.0025, alpha=0.8)
model.train()
model.fit(X=train_data, y=train_labels, n_epochs=1000, loss_fn=loss_fn, optimizer=optimizer)

Epoch 0:	 loss(best per batches): 0.743209719657898 accuracy(best per batch): 0.3999999761581421
Epoch 1:	 loss(best per batches): 0.7177824378013611 accuracy(best per batch): 0.3999999761581421
Epoch 2:	 loss(best per batches): 0.7134370803833008 accuracy(best per batch): 0.3999999761581421
Epoch 3:	 loss(best per batches): 0.710369348526001 accuracy(best per batch): 0.3999999761581421
Epoch 4:	 loss(best per batches): 0.7076522707939148 accuracy(best per batch): 0.3999999761581421
Epoch 5:	 loss(best per batches): 0.7050005793571472 accuracy(best per batch): 0.3999999761581421
Epoch 6:	 loss(best per batches): 0.7026171684265137 accuracy(best per batch): 0.3999999761581421
Epoch 7:	 loss(best per batches): 0.7004379034042358 accuracy(best per batch): 0.3999999761581421
Epoch 8:	 loss(best per batches): 0.6983076930046082 accuracy(best per batch): 0.3999999761581421
Epoch 9:	 loss(best per batches): 0.6965562105178833 accuracy(best per batch): 0.3999999761581421
Epoch 10:	 loss(best p

In [46]:
model.eval()
pred = model.forward(test_data)

print(pred.reshape(1,-1))
print(test_labels.reshape(1,-1))

loss = nn.BCELoss(pred, test_labels)
loss


tensor([[0.0122, 0.9505, 0.3320, 0.0122, 0.3330, 0.0122, 0.9510, 0.9559, 0.0122,
         0.9508, 0.9585, 0.0122, 0.8678, 0.2151, 0.9582, 0.9526, 0.0128, 0.0122,
         0.3330, 0.1709, 0.9578, 0.9576, 0.9554, 0.9568, 0.0125, 0.9568, 0.9566,
         0.9569, 0.9573, 0.9561, 0.8808, 0.3690, 0.9352, 0.8397, 0.9496, 0.3325,
         0.0122, 0.9568, 0.9562, 0.8893, 0.0122, 0.9574, 0.9525, 0.9581, 0.0122,
         0.3322, 0.9563, 0.3323, 0.9574, 0.0122, 0.9542, 0.9571, 0.9565, 0.5550,
         0.3579, 0.9570, 0.9581, 0.0122, 0.5003, 0.9560, 0.9564, 0.0122, 0.0122]],
       device='cuda:0', grad_fn=<ReshapeAliasBackward0>)
tensor([[0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
         0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
         0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1.,
         1., 1., 1., 0., 1., 0., 0., 0., 0.]], device='cuda:0')


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [ ]:
from torchsummary import summary
#summary(model=model, input_size=(None,model.input_size))